In [1]:
# Will Hollingsworth, Colton Murray, Alexander Shiveley

In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Generating Datasets

In [21]:
decision_lambda = lambda x1, x2: 1 if (x1 + (3 * x2) - 2) > 0 else -1

def gen_data():
    """
    Generates random data points and classifies them.
    """
    data = np.random.randint(low=-40, high=40, size=(100,2))
    classification = np.array([[decision_lambda(d[0], d[1])] for d in data])

    return np.hstack((data, classification))

In [22]:
def get_sets(data, split):
    """
    Convenience function that randomly selects a training and test set from the input data.
    
    :param data: (ndarray) the data you want to split
    :param split: (float) the percentage of the data you want to be TRAINING data
    
    :returns: (tuple) a tuple where the first element is the training set, and the second element is the test set
    """
    # Randomly shuffle the order from a copy of the data
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    row_count = data.shape[0]

    # calc the number of samples, assumes the input samples are seperated by row
    training_count = round(row_count * split)
    
    training_set = shuffled[:training_count]
    test_set = shuffled[training_count:]
    
    return training_set, test_set

In [23]:
def split_pos_neg(data):
    """
    Returns two sets of positive, then negative examples (1's then -1's from the output column)
    """
    return data[data[:, -1]==1, :-1], data[data[:, -1]==-1, :-1]

In [24]:
# Find a balanced dataset of positive and negative samples
ratio = -1
while ratio == -1 or abs(1 - ratio) > 0.1:
    data = gen_data()
    pos, neg = split_pos_neg(data)

    # Check ratio of positive and negative samples
    ratio = len(pos) / len(neg)
print(ratio)

1.0


# Perceptron (2 Input Linear Unit)

In [201]:
class linear_unit():
    def __init__(self, num_inputs):
        self.weights = np.random.rand(num_inputs + 1)
        
    def output(self, data_point):
        """
        Returns the linear combination of the input and this unit's weights
        """
        # total = w0*1 + w1x1 + w2x2 + ...
        data_with_bias = np.hstack((np.array([1]), data_point[:-1]))
        t = data_with_bias * self.weights
        t = np.sum(t)
        
        return t
        
    def delta_rule(self, learning_rate, epochs ):
        """
        Performs the delta rule training for a given number of epochs
        """
        # From NN_MitchelChapter4-1.pdf on canvas          
        for ep in range(epochs): 
            delta_w = np.zeros(self.weights.shape[0])
            for d in data:
                d_with_bias = np.hstack((np.array([1]), d[:-1]))
                delta_w = delta_w + learning_rate * (d[2] - self.output(d)) * d_with_bias     
            self.weights = self.weights + delta_w

In [202]:
p = linear_unit(2)

In [203]:
print(data[0])
print(p.weights)
print(p.output(data[0]))

# Train perceptron
p.delta_rule(0.001, 25)
print(p.weights)
print(p.output(data[0]))

[-31  -4  -1]
[0.03801704 0.49188861 0.54372551]
-17.385431935601428
[-2.65149965e+41  4.04923959e+43 -4.10339795e+43]
-1.09139350566825e+45


In [ ]:
# Delta Rule may not be working yet? weights go to high extremes.
# I think optimal weights should approach [-2, 1, 3]. or it gets stu